In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

from utils import data_preprocessing_util as dpu
from utils import classification_util as cu

from MLP import MLP
from nn_framework import NNFramework

import seaborn as sns
import matplotlib.pyplot as plt

# Fertility Dataset

## Preprocessing

In [4]:
data_path = r'data/fertility_diagnosis.csv'
df = pd.read_csv(data_path)
df = dpu.preprocess_fertility_dataset(df)

In [5]:
nn = NNFramework()
nn.fit_encoder(df=df, cols_to_encode=df.columns.difference(['age', 'hours_sitting']))
df_encoded = nn.encode_dataset(df)

In [6]:
len(df_encoded.columns)

21

## Parameter experiments

In [7]:
random_seed = 1038
scaling = True
oversampling = True

scaler = preprocessing.StandardScaler() if scaling else None

In [8]:
y = df_encoded['diagnosis']
X = df_encoded[df_encoded.columns.difference(['diagnosis'])]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_seed, shuffle=True, stratify=y)


Params to check out:
- 2 activation functions
- 3 lrs: 0.001, 0.01, 0.1
- number of nodes per layer
- 1 layer:
    - 5
    - 32
- 2 layer:
    - 16 16
- 3 layers
    - 10 5 5
    - 16 8 8
    - 64 32 32

In [43]:
activation_functions = ['relu', 'sigmoid']
learning_rates = [0.001, 0.01, 0.1]
hidden_layer_sizes = [(5,), (32,), (16, 16), (10, 5, 5), (16, 8, 8), (64, 32, 32),]

In [49]:
methods = []

for af in activation_functions:
    for lr in learning_rates:
        for hls in hidden_layer_sizes:
            methods.append((f'MLP-{af}-{lr}-{hls}', MLP(n_iter=1000, activation_function=af, learning_rate=lr, hidden_layer_sizes=hls)))
    
pipelines = cu.define_pipelines(methods, scaler=scaler, oversampling=oversampling)

In [41]:
import warnings
warnings.filterwarnings("ignore")

In [50]:
cv_num = 5
model_params = {}
models = {}

for model_name, pipeline in pipelines.items():
    cv_results = cross_validate(pipeline, X, y, cv=cv_num, scoring='f1_macro', return_estimator=True)

    models[model_name] = cv_results['estimator']
    model_params[model_name] = {}

    num_cols = ['test_score', 'fit_time', 'score_time']

    for num_col in num_cols:
        model_params[model_name][num_col] = cv_results[num_col]
        model_params[model_name][f'{num_col}_mean'] = cv_results[num_col].mean()
        model_params[model_name][f'{num_col}_std'] = cv_results[num_col].std()
    
    model_params[model_name]['parameter_num'] = cv_results['estimator'][0][model_name].number_of_params_
    model_params[model_name]['hidden_layer_sizes'] = cv_results['estimator'][0][model_name].hidden_layer_sizes
    model_params[model_name]['activation_function'] = cv_results['estimator'][0][model_name].activation_function
    model_params[model_name]['learning_rate'] = cv_results['estimator'][0][model_name].learning_rate
    model_params[model_name]['converged'] = [e[model_name].converged_ for e in cv_results['estimator']]

    print(model_name)
    print(
        f"f1 scores: {model_params[model_name]['test_score']}\n" +
        f"f1 mean: {model_params[model_name]['test_score_mean']:.3f}\n" +
        f"f1 std: {model_params[model_name]['test_score_std']:.3f}\n"
    )
    print('----------------------------------------------------------------------------------------------------')
    

MLP-relu-0.001-(5,)
f1 scores: [0.39393939 0.56709957 0.60784314 0.49820789 0.44444444]
f1 mean: 0.502
f1 std: 0.078

----------------------------------------------------------------------------------------------------
MLP-relu-0.001-(32,)
f1 scores: [0.49820789 0.56709957 0.72222222 0.42857143 0.60784314]
f1 mean: 0.565
f1 std: 0.100

----------------------------------------------------------------------------------------------------
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
MLP-relu-0.001-(16, 16)
f1 scores: [0.09090909 0.09090909 0.09090909 0.41176471 0.39393939]
f1 mean: 0.216
f1 std: 0.153

----------------------------------------------------------------------------------------------------
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized we

In [48]:
cv_results['estimator'][0][model_name].converged_

AttributeError: 'MLP' object has no attribute 'converged'

In [27]:
df_param = pd.DataFrame(model_params).transpose()
df_param = df_param.reset_index(drop=False)
df_param = df_param.rename(columns={'index': 'model'})

# df_param['activation_function'] = df_param.model.str.extract(r'MLP-(\w+)-.*')
# df_param['layer_num'] = df_param.model.str.extract(r'MLP-\w+-(\d+).*').astype(int)
# df_param['layer_sizes'] = df_param.model.str.extract(r'MLP-\w+-\d-(\d+)').astype(int)

df_param

,model,test_score,test_score_mean,test_score_std,fit_time,fit_time_mean,fit_time_std,score_time,score_time_mean,score_time_std,parameter_num,hidden_layer_sizes,activation_function,layer_num,layer_sizes
0,MLP-relu-1-5,"[0.31034482758620685, 0.5670995670995671, 0.49...",0.447992,0.085067,"[0.588165283203125, 0.5871419906616211, 0.5853...",0.582532,0.006307,"[0.0030002593994140625, 0.0020008087158203125,...",0.002401,0.00049,117,"(5,)",relu,1,5
1,MLP-sigmoid-1-5,"[0.34065934065934067, 0.6571428571428571, 0.60...",0.478482,0.128693,"[0.6839420795440674, 0.6629798412322998, 0.674...",0.676983,0.013858,"[0.0019998550415039062, 0.002000093460083008, ...",0.0022,0.0004,117,"(5,)",sigmoid,1,5


Plots to do:

## Comparison 